In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# ===== Load Data =====
folder_path = Path("data")  # Make sure there's a 'data/' folder next to this script

building_years = []
for year in range(2015, 2020):
    df = pd.read_csv(folder_path / f'building_details_{year}.csv')  # FIXED
    df['year'] = year
    building_years.append(df)

building_all = pd.concat(building_years, ignore_index=True)
building_all = building_all.drop_duplicates(subset=['acct', 'year'], keep='first')
pivoted = building_all.pivot(index='acct', columns='year')
pivoted.columns = [f'{col}_{year}' for col, year in pivoted.columns]
pivoted = pivoted.reset_index()

train = pd.read_csv(folder_path / 'assessment_history_train.csv')  # FIXED
test = pd.read_csv(folder_path / 'assessment_history_test.csv')    # FIXED

train_merged = train.merge(pivoted, on='acct', how='left')
test_merged = test.merge(pivoted, on='acct', how='left')


/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2274541461.py:18: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + f'building_details_{year}.csv')
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2274541461.py:30: DtypeWarning: Columns (21,23,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(folder_path + 'assessment_history_test.csv')


In [ ]:
# Step 1: Identify all '2019' columns in train
train_2019_cols = [col for col in train_merged.columns if '2019' in col]

# Step 2: Find which 2019 columns are missing in test
train_only_2019_cols = [col for col in train_2019_cols if col not in test_merged.columns]

# Step 3: Always drop 'protested_2019' due to leakage risk
train_only_2019_cols.append('protested_2019')

# Step 4: Drop the identified columns from train
train_merged = train_merged.drop(columns=train_only_2019_cols, errors='ignore')

# Step 5: Optional logging
print(f"Dropped {len(train_only_2019_cols)} columns from train (including protested_2019 if present):")
print(train_only_2019_cols)

✅ Dropped 4 columns from train (including protested_2019 if present):
['building_value_2019', 'land_value_2019', 'assessed_2019', 'protested_2019']


In [ ]:
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    if 'protested_2019' in df.columns:
        df.drop(columns='protested_2019', inplace=True)
        print(f"Dropped 'protested_2019' from {df_name}")
    else:
        print(f"'protested_2019' not found in {df_name}")


ℹ️ 'protested_2019' not found in train_merged
✅ Dropped 'protested_2019' from test_merged


In [ ]:
## Get all '2019' columns in train and test
train_2019_cols = sorted([col for col in train_merged.columns if '2019' in col])
test_2019_cols = sorted([col for col in test_merged.columns if '2019' in col])

# Print train columns
print("2019 Columns in train_merged:")
for col in train_2019_cols:
    print(f" - {col}")

print("\n 2019 Columns in test_merged:")
for col in test_2019_cols:
    print(f" - {col}")


📘 2019 Columns in train_merged:
 - bedrooms_2019
 - building_area_2019
 - building_condition_2019
 - deck_area_2019
 - elevator_2019
 - exterior_walls_2019
 - fireplaces_2019
 - floor_area_lower_2019
 - floor_area_primary_2019
 - floor_area_upper_2019
 - floors_2019
 - foundation_type_2019
 - full_bath_2019
 - garage_area_2019
 - grade_2019
 - half_bath_2019
 - has_cooling_2019
 - has_heat_2019
 - land_area_2019
 - mobile_home_area_2019
 - physical_condition_2019
 - porch_area_2019
 - quality_2019
 - quality_description_2019
 - total_rooms_2019
 - year_built_2019
 - year_remodeled_2019

📙 2019 Columns in test_merged:
 - bedrooms_2019
 - building_area_2019
 - building_condition_2019
 - deck_area_2019
 - elevator_2019
 - exterior_walls_2019
 - fireplaces_2019
 - floor_area_lower_2019
 - floor_area_primary_2019
 - floor_area_upper_2019
 - floors_2019
 - foundation_type_2019
 - full_bath_2019
 - garage_area_2019
 - grade_2019
 - half_bath_2019
 - has_cooling_2019
 - has_heat_2019
 - land_a

In [ ]:
# Compare sets
train_only_2019 = sorted(list(set(train_2019_cols) - set(test_2019_cols)))
test_only_2019 = sorted(list(set(test_2019_cols) - set(train_2019_cols)))

# Print comparison result
if not train_only_2019 and not test_only_2019:
    print("\n All 2019 columns match between train_merged and test_merged.")
else:
    print("\n Mismatched 2019 columns found:")

    if train_only_2019:
        print(" In train_merged but not in test_merged:")
        for col in train_only_2019:
            print(f"   - {col}")

    if test_only_2019:
        print(" In test_merged but not in train_merged:")
        for col in test_only_2019:
            print(f"   - {col}")


✅ All 2019 columns match between train_merged and test_merged.


In [9]:
# Step 1: Store target separately
y_train = train_merged["TARGET"].values

# Step 4: Confirm sizes
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")


Train shape: (628287, 37)
Test shape: (418858, 33)


In [10]:
# Calculate % of missing values in each column of TRAIN only
train_na = (train_merged.isnull().sum() / train_merged.shape[0]) * 100

# Drop columns with no missing values, sort the rest
train_na = train_na[train_na > 0].sort_values(ascending=False)

# Optional: Show top 200 missing features
missing_data = pd.DataFrame({'Missing Ratio (%)': train_na})

# Display
print("Top columns with missing data (based on training set only):")
missing_data.head(20)



Top columns with missing data (based on training set only):


,Missing Ratio (%)
building_area_2015,5.954126
land_area_2015,5.954126
protested_2015,5.954126
assessed_2015,5.954126
building_value_2015,5.954126
land_value_2015,5.954126
quality_description_2015,5.742280
quality_2015,5.742280
year_remodeled_2015,5.742280
year_built_2015,5.742280


In [ ]:
def create_floor_area_totals(df, years):
    for y in years:
        primary_col = f'floor_area_primary_{y}'
        upper_col   = f'floor_area_upper_{y}'
        lower_col   = f'floor_area_lower_{y}'
        total_col   = f'floor_area_total_{y}'

        if all(col in df.columns for col in [primary_col, upper_col, lower_col]):
            df[total_col] = (
                df[primary_col].fillna(0) +
                df[upper_col].fillna(0) +
                df[lower_col].fillna(0)
            )
            print(f" Created: {total_col}")
        else:
            print(f" Skipping {total_col} — one or more components missing.")
    return df

# Apply for years 2015–2019
years = ['2015', '2016', '2017', '2018', '2019']
train_merged = create_floor_area_totals(train_merged, years)
test_merged = create_floor_area_totals(test_merged, years)



✅ Created: floor_area_total_2015
✅ Created: floor_area_total_2016
✅ Created: floor_area_total_2017
✅ Created: floor_area_total_2018
✅ Created: floor_area_total_2019
✅ Created: floor_area_total_2015
✅ Created: floor_area_total_2016
✅ Created: floor_area_total_2017
✅ Created: floor_area_total_2018
✅ Created: floor_area_total_2019


In [ ]:
zero_pct = (train_merged['mobile_home_area_2015'] == 0).mean() * 100
print(f" mobile_home_area_2015 is 0 in {zero_pct:.2f}% of training rows")


📊 mobile_home_area_2015 is 0 in 94.42% of training rows


In [ ]:
cols_to_drop = [col for col in train_merged.columns if col.startswith("mobile_home_area")]

# Drop from both sets
train_merged.drop(columns=cols_to_drop, inplace=True)
test_merged.drop(columns=cols_to_drop, inplace=True)

print(f" Dropped columns from train/test: {cols_to_drop}")



🗑️ Dropped columns from train/test: ['mobile_home_area_2015', 'mobile_home_area_2016', 'mobile_home_area_2017', 'mobile_home_area_2018', 'mobile_home_area_2019']


In [ ]:
zero_pct = (train_merged['deck_area_2015'] == 0).mean() * 100
print(f" deck_area_2015 is 0 in {zero_pct:.2f}% of training rows")



📊 deck_area_2015 is 0 in 90.66% of training rows


In [ ]:
# Use training columns to identify which deck_area columns exist
cols_to_drop = [col for col in train_merged.columns if col.startswith("deck_area")]

# Drop those columns from both datasets
train_merged.drop(columns=cols_to_drop, inplace=True)
test_merged.drop(columns=cols_to_drop, inplace=True)

print(f" Dropped columns from train/test: {cols_to_drop}")



🗑️ Dropped columns from train/test: ['deck_area_2015', 'deck_area_2016', 'deck_area_2017', 'deck_area_2018', 'deck_area_2019']


In [ ]:
def backfill_yearly_features(df, base_features, years):
    for feature in base_features:
        cols = [f"{feature}_{y}" for y in years if f"{feature}_{y}" in df.columns]
        if len(cols) >= 2:
            # Backfill all relevant year columns in-place
            df[cols] = df[cols].bfill(axis=1)
            print(f" Backfilled: {feature} across {cols}")
        else:
            print(f" Skipping {feature} — not enough year columns found.")
    return df

# Years in reverse so that bfill works from most recent (2019) to oldest (2015)
years = ['2019', '2018', '2017', '2016', '2015']
features = ['garage_area', 'porch_area']

# Apply to both train and test
train_merged = backfill_yearly_features(train_merged, features, years)
test_merged = backfill_yearly_features(test_merged, features, years)


✅ Backfilled: garage_area across ['garage_area_2019', 'garage_area_2018', 'garage_area_2017', 'garage_area_2016', 'garage_area_2015']
✅ Backfilled: porch_area across ['porch_area_2019', 'porch_area_2018', 'porch_area_2017', 'porch_area_2016', 'porch_area_2015']
✅ Backfilled: garage_area across ['garage_area_2019', 'garage_area_2018', 'garage_area_2017', 'garage_area_2016', 'garage_area_2015']
✅ Backfilled: porch_area across ['porch_area_2019', 'porch_area_2018', 'porch_area_2017', 'porch_area_2016', 'porch_area_2015']


In [ ]:
def backfill_yearly_features(df, features, years):
    for feature in features:
        year_cols = [f"{feature}_{y}" for y in years if f"{feature}_{y}" in df.columns]
        if len(year_cols) >= 2:
            df[year_cols] = df[year_cols].bfill(axis=1)
            print(f" Backfilled: {feature} across {year_cols}")
        else:
            print(f" Skipped: Not enough year columns for '{feature}'")
    return df

# Use years in reverse for proper backfill (newest to oldest)
years = ['2019', '2018', '2017', '2016', '2015']
features = ['floors', 'half_bath', 'full_bath', 'total_rooms', 'bedrooms']

# Apply to both train and test
train_merged = backfill_yearly_features(train_merged, features, years)
test_merged = backfill_yearly_features(test_merged, features, years)




✅ Backfilled: floors across ['floors_2019', 'floors_2018', 'floors_2017', 'floors_2016', 'floors_2015']
✅ Backfilled: half_bath across ['half_bath_2019', 'half_bath_2018', 'half_bath_2017', 'half_bath_2016', 'half_bath_2015']
✅ Backfilled: full_bath across ['full_bath_2019', 'full_bath_2018', 'full_bath_2017', 'full_bath_2016', 'full_bath_2015']
✅ Backfilled: total_rooms across ['total_rooms_2019', 'total_rooms_2018', 'total_rooms_2017', 'total_rooms_2016', 'total_rooms_2015']
✅ Backfilled: bedrooms across ['bedrooms_2019', 'bedrooms_2018', 'bedrooms_2017', 'bedrooms_2016', 'bedrooms_2015']
✅ Backfilled: floors across ['floors_2019', 'floors_2018', 'floors_2017', 'floors_2016', 'floors_2015']
✅ Backfilled: half_bath across ['half_bath_2019', 'half_bath_2018', 'half_bath_2017', 'half_bath_2016', 'half_bath_2015']
✅ Backfilled: full_bath across ['full_bath_2019', 'full_bath_2018', 'full_bath_2017', 'full_bath_2016', 'full_bath_2015']
✅ Backfilled: total_rooms across ['total_rooms_2019', 

In [18]:
# Identify elevator-related columns from training set
elevator_cols = [col for col in train_merged.columns if col.startswith("elevator")]

# Drop from both train and test
train_merged.drop(columns=elevator_cols, inplace=True)
test_merged.drop(columns=elevator_cols, inplace=True)

print(f"🗑️ Dropped elevator-related columns from train/test: {elevator_cols}")


🗑️ Dropped elevator-related columns from train/test: ['elevator_2015', 'elevator_2016', 'elevator_2017', 'elevator_2018', 'elevator_2019']


In [ ]:
def backfill_year_features(df, features, years):
    for feature in features:
        cols = [f"{feature}_{y}" for y in years if f"{feature}_{y}" in df.columns]
        if len(cols) >= 2:
            df[cols] = df[cols].bfill(axis=1)
            print(f" Backfilled: {feature} across {cols}")
        else:
            print(f" Skipped {feature} — not enough year-based columns found.")
    return df

def create_year_built_final(df):
    year_cols = [f"year_built_{y}" for y in ['2019', '2018', '2017', '2016', '2015'] if f"year_built_{y}" in df.columns]
    if year_cols:
        df['year_built_final'] = df[year_cols].bfill(axis=1).iloc[:, 0]
        print(f" Created year_built_final from: {year_cols}")
    else:
        print(" Skipped: no year_built_* columns found.")
    return df

# Define reverse years for backfill (latest → oldest)
years = ['2019', '2018', '2017', '2016', '2015']
features_to_backfill = ['fireplaces', 'quality', 'quality_description']

# Apply to both train and test
train_merged = backfill_year_features(train_merged, features_to_backfill, years)
train_merged = create_year_built_final(train_merged)

test_merged = backfill_year_features(test_merged, features_to_backfill, years)
test_merged = create_year_built_final(test_merged)


✅ Backfilled: fireplaces across ['fireplaces_2019', 'fireplaces_2018', 'fireplaces_2017', 'fireplaces_2016', 'fireplaces_2015']
✅ Backfilled: quality across ['quality_2019', 'quality_2018', 'quality_2017', 'quality_2016', 'quality_2015']
✅ Backfilled: quality_description across ['quality_description_2019', 'quality_description_2018', 'quality_description_2017', 'quality_description_2016', 'quality_description_2015']
✅ Created year_built_final from: ['year_built_2019', 'year_built_2018', 'year_built_2017', 'year_built_2016', 'year_built_2015']
✅ Backfilled: fireplaces across ['fireplaces_2019', 'fireplaces_2018', 'fireplaces_2017', 'fireplaces_2016', 'fireplaces_2015']
✅ Backfilled: quality across ['quality_2019', 'quality_2018', 'quality_2017', 'quality_2016', 'quality_2015']
✅ Backfilled: quality_description across ['quality_description_2019', 'quality_description_2018', 'quality_description_2017', 'quality_description_2016', 'quality_description_2015']
✅ Created year_built_final from

In [20]:
## Identify columns to drop from training data
cols_to_drop = [col for col in train_merged.columns if col.startswith("year_remodeled")]

# Drop from both train and test
train_merged.drop(columns=cols_to_drop, inplace=True)
test_merged.drop(columns=cols_to_drop, inplace=True)

print(f"🗑️ Dropped year_remodeled-related columns from train/test: {cols_to_drop}")



🗑️ Dropped year_remodeled-related columns from train/test: ['year_remodeled_2015', 'year_remodeled_2016', 'year_remodeled_2017', 'year_remodeled_2018', 'year_remodeled_2019']


In [ ]:
def backfill_categorical_year_features(df, features, years):
    for feature in features:
        year_cols = [f"{feature}_{y}" for y in years if f"{feature}_{y}" in df.columns]
        if len(year_cols) >= 2:
            df[year_cols] = df[year_cols].bfill(axis=1)
            print(f" Backfilled: {feature} across {year_cols}")
        else:
            print(f" Skipped: {feature} — not enough year-based columns.")
    return df

# Backfill from most recent year to oldest
years = ['2019', '2018', '2017', '2016', '2015']
features = ['building_condition', 'foundation_type', 'grade', 'has_cooling', 
            'has_heat', 'physical_condition', 'exterior_walls']

# Apply to train and test
train_merged = backfill_categorical_year_features(train_merged, features, years)
test_merged = backfill_categorical_year_features(test_merged, features, years)



✅ Backfilled: building_condition across ['building_condition_2019', 'building_condition_2018', 'building_condition_2017', 'building_condition_2016', 'building_condition_2015']
✅ Backfilled: foundation_type across ['foundation_type_2019', 'foundation_type_2018', 'foundation_type_2017', 'foundation_type_2016', 'foundation_type_2015']
✅ Backfilled: grade across ['grade_2019', 'grade_2018', 'grade_2017', 'grade_2016', 'grade_2015']
✅ Backfilled: has_cooling across ['has_cooling_2019', 'has_cooling_2018', 'has_cooling_2017', 'has_cooling_2016', 'has_cooling_2015']
✅ Backfilled: has_heat across ['has_heat_2019', 'has_heat_2018', 'has_heat_2017', 'has_heat_2016', 'has_heat_2015']
✅ Backfilled: physical_condition across ['physical_condition_2019', 'physical_condition_2018', 'physical_condition_2017', 'physical_condition_2016', 'physical_condition_2015']
✅ Backfilled: exterior_walls across ['exterior_walls_2019', 'exterior_walls_2018', 'exterior_walls_2017', 'exterior_walls_2016', 'exterior_wal

In [ ]:
def backfill_year_columns(df, features, years):
    for feature in features:
        cols = [f"{feature}_{y}" for y in years if f"{feature}_{y}" in df.columns]
        if len(cols) >= 2:
            df[cols] = df[cols].bfill(axis=1)
            print(f" Backfilled: {feature} across {cols}")
        else:
            print(f" Skipped: {feature} — not enough year-based columns found.")
    return df

# Define year ranges
area_years = ['2019', '2018', '2017', '2016', '2015']
value_years = ['2018', '2017', '2016', '2015']

# Define feature groups
area_features = ['building_area', 'land_area']
value_features = ['building_value', 'land_value', 'assessed']

# Apply to train and test
train_merged = backfill_year_columns(train_merged, area_features, area_years)
train_merged = backfill_year_columns(train_merged, value_features, value_years)

test_merged = backfill_year_columns(test_merged, area_features, area_years)
test_merged = backfill_year_columns(test_merged, value_features, value_years)


✅ Backfilled: building_area across ['building_area_2019', 'building_area_2018', 'building_area_2017', 'building_area_2016', 'building_area_2015']
✅ Backfilled: land_area across ['land_area_2019', 'land_area_2018', 'land_area_2017', 'land_area_2016', 'land_area_2015']
✅ Backfilled: building_value across ['building_value_2018', 'building_value_2017', 'building_value_2016', 'building_value_2015']
✅ Backfilled: land_value across ['land_value_2018', 'land_value_2017', 'land_value_2016', 'land_value_2015']
✅ Backfilled: assessed across ['assessed_2018', 'assessed_2017', 'assessed_2016', 'assessed_2015']
✅ Backfilled: building_area across ['building_area_2019', 'building_area_2018', 'building_area_2017', 'building_area_2016', 'building_area_2015']
✅ Backfilled: land_area across ['land_area_2019', 'land_area_2018', 'land_area_2017', 'land_area_2016', 'land_area_2015']
✅ Backfilled: building_value across ['building_value_2018', 'building_value_2017', 'building_value_2016', 'building_value_2015'

In [ ]:
def backfill_protested_columns(df, years):
    cols = [f"protested_{y}" for y in years if f"protested_{y}" in df.columns]
    if len(cols) >= 2:
        df[cols] = df[cols].bfill(axis=1)
        print(f" Backfilled: protested across {cols}")
    else:
        print(" Skipped protested — not enough year-based columns found.")
    return df

# Only use pre-2019 years to avoid leakage
years = ['2018', '2017', '2016', '2015']

# Apply to both datasets
train_merged = backfill_protested_columns(train_merged, years)
test_merged = backfill_protested_columns(test_merged, years)



✅ Backfilled: protested across ['protested_2018', 'protested_2017', 'protested_2016', 'protested_2015']
✅ Backfilled: protested across ['protested_2018', 'protested_2017', 'protested_2016', 'protested_2015']


In [24]:
# Save 'acct' from test_merged only
acct_test = test_merged[['acct']].copy() if 'acct' in test_merged.columns else None

# Drop 'acct' from both train and test
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    if 'acct' in df.columns:
        df.drop(columns='acct', inplace=True)
        print(f"🗑️ Dropped 'acct' from {df_name}")




🗑️ Dropped 'acct' from train_merged
🗑️ Dropped 'acct' from test_merged


In [25]:
for col in ['zone', 'subneighborhood', 'neighborhood', 'region']:
    if col in train_merged.columns:
        print(f"{col}: {train_merged[col].nunique()} unique values in training set")
    else:
        print(f"⚠️ {col} not found in training set")


zone: 1589 unique values in training set
subneighborhood: 6161 unique values in training set
neighborhood: 959 unique values in training set
region: 87 unique values in training set


In [ ]:
for col in ['neighborhood', 'region','zone','subneighborhood']:
    if col in train_merged.columns:
        # Step 1: Compute frequency from training data
        freq_map = train_merged[col].value_counts(normalize=True)

        # Step 2: Apply to both datasets
        train_merged[f'{col}_freq'] = train_merged[col].map(freq_map)
        test_merged[f'{col}_freq'] = test_merged[col].map(freq_map)

        print(f" Frequency encoded: {col} → {col}_freq (based on training set)")
    else:
        print(f" Column '{col}' not found in training set")


/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2858503920.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_merged[f'{col}_freq'] = train_merged[col].map(freq_map)
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2858503920.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_merged[f'{col}_freq'] = test_merged[col].map(freq_map)
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2858503920.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

✅ Frequency encoded: neighborhood → neighborhood_freq (based on training set)
✅ Frequency encoded: region → region_freq (based on training set)


/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2858503920.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_merged[f'{col}_freq'] = train_merged[col].map(freq_map)
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2858503920.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_merged[f'{col}_freq'] = test_merged[col].map(freq_map)


✅ Frequency encoded: zone → zone_freq (based on training set)
✅ Frequency encoded: subneighborhood → subneighborhood_freq (based on training set)


/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2858503920.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_merged[f'{col}_freq'] = train_merged[col].map(freq_map)
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2858503920.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_merged[f'{col}_freq'] = test_merged[col].map(freq_map)


In [ ]:
if all(col in train_merged.columns for col in ['land_area_2019', 'land_area_2015']):
    growth = train_merged['land_area_2019'] - train_merged['land_area_2015']
    changed_pct = (growth != 0).mean() * 100
    print(f" Percentage of homes with land area change (2015→2019): {changed_pct:.2f}%")
else:
    print(" One or both columns ('land_area_2015', 'land_area_2019') not found in training set")



🧪 Percentage of homes with land area change (2015→2019): 6.42%


In [ ]:
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    if 'year_built_final' in df.columns:
        df['year_built_final'] = df['year_built_final'].astype(str)
        print(f" Converted 'year_built_final' to string in {df_name}")
    else:
        print(f" 'year_built_final' not found in {df_name}")


🔤 Converted 'year_built_final' to string in train_merged
🔤 Converted 'year_built_final' to string in test_merged


In [ ]:
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    if 'floor_area_total_final' in df.columns and 'year_built_final' in df.columns:
        df.loc[df['floor_area_total_final'] == 0, 'year_built_final'] = 'None'
        print(f" Set 'year_built_final' to 'None' where 'floor_area_total_final' == 0 in {df_name}")
    else:
        print(f" Required columns missing in {df_name}")


⚠️ Required columns missing in train_merged
⚠️ Required columns missing in test_merged


In [ ]:
# Define year ranges per feature type
value_years = range(2015, 2019)   # 2015–2018 for value columns
area_years  = range(2015, 2020)   # 2015–2019 for area columns

# Define base columns
base_cols_year_map = {
    'building_value': value_years,
    'land_value': value_years,
    'building_area': area_years,
    'land_area': area_years,
}

# Generate full list of columns to impute
cols_to_impute = []
for base, years in base_cols_year_map.items():
    for year in years:
        cols_to_impute.append(f"{base}_{year}")

# Impute using neighborhood → region strategy
for col in cols_to_impute:
    # First, fill by neighborhood (train only)
    if 'neighborhood' in train_merged.columns:
        medians_by_neigh = train_merged.groupby('neighborhood')[col].median()
        train_merged[col] = train_merged.apply(
            lambda row: medians_by_neigh[row['neighborhood']] if pd.isna(row[col]) else row[col], axis=1)
        test_merged[col] = test_merged.apply(
            lambda row: medians_by_neigh.get(row['neighborhood'], np.nan) if pd.isna(row[col]) else row[col], axis=1)

    # Then, fill remaining by region (train only)
    if 'region' in train_merged.columns:
        medians_by_region = train_merged.groupby('region')[col].median()
        train_merged[col] = train_merged.apply(
            lambda row: medians_by_region[row['region']] if pd.isna(row[col]) else row[col], axis=1)
        test_merged[col] = test_merged.apply(
            lambda row: medians_by_region.get(row['region'], np.nan) if pd.isna(row[col]) else row[col], axis=1)

    print(f" Imputed '{col}' using group medians (neighborhood → region) from training data")



✅ Imputed 'building_value_2015' using group medians (neighborhood → region) from training data
✅ Imputed 'building_value_2016' using group medians (neighborhood → region) from training data
✅ Imputed 'building_value_2017' using group medians (neighborhood → region) from training data
✅ Imputed 'building_value_2018' using group medians (neighborhood → region) from training data
✅ Imputed 'land_value_2015' using group medians (neighborhood → region) from training data
✅ Imputed 'land_value_2016' using group medians (neighborhood → region) from training data
✅ Imputed 'land_value_2017' using group medians (neighborhood → region) from training data
✅ Imputed 'land_value_2018' using group medians (neighborhood → region) from training data
✅ Imputed 'building_area_2015' using group medians (neighborhood → region) from training data
✅ Imputed 'building_area_2016' using group medians (neighborhood → region) from training data
✅ Imputed 'building_area_2017' using group medians (neighborhood → r

In [ ]:
from pandas.api.types import is_numeric_dtype

# Step 1: Coerce non-numeric to NaN in both sets
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    if 'year_built_final' in df.columns:
        df['year_built_final'] = pd.to_numeric(df['year_built_final'], errors='coerce')
        print(f" Coerced 'year_built_final' to numeric in {df_name}")

# Step 2: Group-based imputation (use training data only)
if 'year_built_final' in train_merged.columns and 'neighborhood' in train_merged.columns:
    # Neighborhood-based median from train
    neigh_medians = train_merged.groupby('neighborhood')['year_built_final'].median()

    # Apply to train
    train_merged['year_built_final'] = train_merged.apply(
        lambda row: neigh_medians[row['neighborhood']] if pd.isna(row['year_built_final']) else row['year_built_final'], axis=1)

    # Apply to test
    test_merged['year_built_final'] = test_merged.apply(
        lambda row: neigh_medians.get(row['neighborhood'], np.nan) if pd.isna(row['year_built_final']) else row['year_built_final'], axis=1)

if 'region' in train_merged.columns:
    # Region-based fallback median from train
    region_medians = train_merged.groupby('region')['year_built_final'].median()

    train_merged['year_built_final'] = train_merged.apply(
        lambda row: region_medians[row['region']] if pd.isna(row['year_built_final']) else row['year_built_final'], axis=1)

    test_merged['year_built_final'] = test_merged.apply(
        lambda row: region_medians.get(row['region'], np.nan) if pd.isna(row['year_built_final']) else row['year_built_final'], axis=1)

print(" Imputed 'year_built_final' using neighborhood → region medians from training set")


🔢 Coerced 'year_built_final' to numeric in train_merged
🔢 Coerced 'year_built_final' to numeric in test_merged
✅ Imputed 'year_built_final' using neighborhood → region medians from training set


In [ ]:
# List of all assessed columns to impute
assessed_cols = ['assessed_2015', 'assessed_2016', 'assessed_2017', 'assessed_2018']

for col in assessed_cols:
    if col not in train_merged.columns:
        continue

    # Step 1: Compute medians from training data only
    neigh_medians = train_merged.groupby('neighborhood')[col].median()
    region_medians = train_merged.groupby('region')[col].median()
    global_median = train_merged[col].median()

    # Step 2: Train set imputation
    train_merged[col] = train_merged.apply(
        lambda row: neigh_medians[row['neighborhood']]
        if pd.isna(row[col]) and row['neighborhood'] in neigh_medians else
        region_medians[row['region']]
        if pd.isna(row[col]) and row['region'] in region_medians else
        global_median
        if pd.isna(row[col]) else
        row[col],
        axis=1
    )

    # Step 3: Test set imputation (using train medians only)
    test_merged[col] = test_merged.apply(
        lambda row: neigh_medians.get(row['neighborhood'], np.nan)
        if pd.isna(row[col]) else row[col],
        axis=1
    )
    test_merged[col] = test_merged.apply(
        lambda row: region_medians.get(row['region'], np.nan)
        if pd.isna(row[col]) else row[col],
        axis=1
    )
    test_merged[col].fillna(global_median, inplace=True)

    print(f" Imputed '{col}' using neighborhood → region → global medians (from training data)")



/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/3523755006.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_merged[col].fillna(global_median, inplace=True)


✅ Imputed 'assessed_2015' using neighborhood → region → global medians (from training data)


/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/3523755006.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_merged[col].fillna(global_median, inplace=True)


✅ Imputed 'assessed_2016' using neighborhood → region → global medians (from training data)


/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/3523755006.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_merged[col].fillna(global_median, inplace=True)


✅ Imputed 'assessed_2017' using neighborhood → region → global medians (from training data)
✅ Imputed 'assessed_2018' using neighborhood → region → global medians (from training data)


/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/3523755006.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_merged[col].fillna(global_median, inplace=True)


In [ ]:
# === Step 1: Compute neighborhood-level stats ===
neigh_stats = train_merged.groupby('neighborhood')['assessed_2018'].agg([
    ('neigh_assess_mean', 'mean'),
    ('neigh_assess_median', 'median'),
    ('neigh_assess_std', 'std'),
    ('neigh_assess_q1', lambda x: x.quantile(0.25)),
    ('neigh_assess_q3', lambda x: x.quantile(0.75)),
]).reset_index()
neigh_stats['neigh_assess_iqr'] = neigh_stats['neigh_assess_q3'] - neigh_stats['neigh_assess_q1']

# === Step 2: Compute region-level stats ===
region_stats = train_merged.groupby('region')['assessed_2018'].agg([
    ('region_assess_mean', 'mean'),
    ('region_assess_median', 'median'),
    ('region_assess_std', 'std'),
    ('region_assess_q1', lambda x: x.quantile(0.25)),
    ('region_assess_q3', lambda x: x.quantile(0.75)),
]).reset_index()
region_stats['region_assess_iqr'] = region_stats['region_assess_q3'] - region_stats['region_assess_q1']

# === Step 3: Fallback std maps from training data ===
# For neighborhood fallback, group region medians of neighborhood std
neigh_std_by_region = neigh_stats.merge(train_merged[['neighborhood', 'region']], on='neighborhood', how='left') \
                                  .groupby('region')['neigh_assess_std'].median()
global_neigh_std = neigh_stats['neigh_assess_std'].median()

region_std_by_neigh = region_stats.merge(train_merged[['neighborhood', 'region']], on='region', how='left') \
                                   .groupby('neighborhood')['region_assess_std'].median()
global_region_std = region_stats['region_assess_std'].median()

# === Step 4: Merge into train/test and compute features ===
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    df = df.merge(neigh_stats, on='neighborhood', how='left')
    df = df.merge(region_stats, on='region', how='left')

    # Fill missing std values via fallback
    df['neigh_assess_std'] = df['neigh_assess_std'].fillna(
        df['region'].map(neigh_std_by_region)
    ).fillna(global_neigh_std)

    df['region_assess_std'] = df['region_assess_std'].fillna(
        df['neighborhood'].map(region_std_by_neigh)
    ).fillna(global_region_std)

    # Compute derived features
    df['assess_minus_neigh_mean'] = df['assessed_2018'] - df['neigh_assess_mean']
    df['assess_ratio_neigh_mean'] = df['assessed_2018'] / (df['neigh_assess_mean'] + 1e-6)
    df['z_score_assess_neigh'] = df['assess_minus_neigh_mean'] / (df['neigh_assess_std'] + 1e-6)

    df['assess_minus_region_mean'] = df['assessed_2018'] - df['region_assess_mean']
    df['assess_ratio_region_mean'] = df['assessed_2018'] / (df['region_assess_mean'] + 1e-6)
    df['z_score_assess_region'] = df['assess_minus_region_mean'] / (df['region_assess_std'] + 1e-6)

    # Save back
    if df_name == 'train_merged':
        train_merged = df
    else:
        test_merged = df

print(" Completed: Stats merge + std fallback + z-score computation.")


✅ Completed: Stats merge + std fallback + z-score computation.


In [34]:
cols_to_drop = ['neighborhood', 'region','zone','subneighborhood']

for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    drop_cols = [col for col in cols_to_drop if col in df.columns]
    if drop_cols:
        df.drop(columns=drop_cols, inplace=True)
        print(f"🗑️ Dropped columns from {df_name}: {drop_cols}")


🗑️ Dropped columns from train_merged: ['neighborhood', 'region', 'zone', 'subneighborhood']
🗑️ Dropped columns from test_merged: ['neighborhood', 'region', 'zone', 'subneighborhood']


In [ ]:
growth_pairs = {
    'building_value_growth': ('building_value_2018', 'building_value_2015'),
    'land_value_growth':     ('land_value_2018', 'land_value_2015'),
    'assessed_growth':       ('assessed_2018', 'assessed_2015')
}

for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    for new_col, (final_col, base_col) in growth_pairs.items():
        if final_col in df.columns and base_col in df.columns:
            df[new_col] = df[final_col] - df[base_col]
            print(f" Created {new_col} in {df_name}")
        else:
            print(f" Skipped {new_col} in {df_name}: missing {final_col} or {base_col}")

    # === Building Age ===
    if 'year_built_final' in df.columns:
        df['year_built_final'] = pd.to_numeric(df['year_built_final'], errors='coerce')  # handle 'Vacant', etc.
        df['building_age'] = 2018 - df['year_built_final']
        print(f" Created building_age in {df_name}")
    else:
        print(f" Skipped building_age in {df_name}: missing year_built_final")


✅ Created building_value_growth in train_merged
✅ Created land_value_growth in train_merged
✅ Created assessed_growth in train_merged
✅ Created building_age in train_merged
✅ Created building_value_growth in test_merged
✅ Created land_value_growth in test_merged
✅ Created assessed_growth in test_merged
✅ Created building_age in test_merged


In [ ]:
if 'school_dist' in train_merged.columns:
    missing_pct = train_merged['school_dist'].isna().mean() * 100
    missing_count = train_merged['school_dist'].isna().sum()

    print(f" 'school_dist' has {missing_count} missing values ({missing_pct:.2f}%) in training set")
else:
    print(" 'school_dist' not found in training set")


🔍 'school_dist' has 3 missing values (0.00%) in training set


In [ ]:
if 'school_dist' in train_merged.columns:
    school_dist_median = train_merged['school_dist'].median()

    for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
        if 'school_dist' in df.columns:
            df['school_dist'] = df['school_dist'].fillna(school_dist_median)
            print(f" Filled missing 'school_dist' in {df_name} using train median ({school_dist_median})")
else:
    print(" 'school_dist' not found in train_merged")


✅ Filled missing 'school_dist' in train_merged using train median (8.0)
✅ Filled missing 'school_dist' in test_merged using train median (8.0)


In [ ]:
if 'year_built_final' in train_merged.columns:
    missing_pct = train_merged['year_built_final'].isna().mean() * 100
    missing_count = train_merged['year_built_final'].isna().sum()

    print(f" 'year_built_final' has {missing_count} missing values ({missing_pct:.2f}%) in training set")
else:
    print(" 'year_built_final' not found in training set")


🔍 'year_built_final' has 0 missing values (0.00%) in training set


In [ ]:
# === STEP 0: Define base feature names ===
numeric_bases = [
    'garage_area', 'porch_area', 'floors', 'half_bath', 'full_bath',
    'total_rooms', 'bedrooms', 'fireplaces', 'building_area', 'building_value'
]

categorical_fill_map = {
    'quality': 'None',
    'quality_description': 'None',
    'building_condition': 'None',
    'foundation_type': 'None',
    'grade': 'None',
    'has_cooling': False,
    'has_heat': False,
    'physical_condition': 'None',
    'exterior_walls': 'None',
    'protested': False
}

# Generate full list of columns (2015–2019 only, no final columns)
numeric_cols_to_zero = [
    f'{base}_{year}' for base in numeric_bases for year in range(2015, 2020)
] + ['building_value_growth']

categorical_cols_to_fill = {
    f'{base}_{year}': val
    for base, val in categorical_fill_map.items()
    for year in range(2015, 2020)
}

# === STEP 1: Apply imputation if floor_area_total_2019 == 0 ===
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    if 'floor_area_total_2019' in df.columns:
        zero_floor_mask = df['floor_area_total_2019'] == 0

        # Fill numeric columns with 0
        for col in numeric_cols_to_zero:
            if col in df.columns:
                df.loc[zero_floor_mask, col] = df.loc[zero_floor_mask, col].fillna(0)

        # Fill categorical/boolean columns
        for col, fill_val in categorical_cols_to_fill.items():
            if col in df.columns:
                df.loc[zero_floor_mask, col] = df.loc[zero_floor_mask, col].fillna(fill_val)

        print(f" Filled structure-dependent missing values in {df_name} for {zero_floor_mask.sum()} rows")
    else:
        print(f" 'floor_area_total_2019' not found in {df_name}")



✅ Filled structure-dependent missing values in train_merged for 218 rows
✅ Filled structure-dependent missing values in test_merged for 150 rows


In [41]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import KFold

# Clear specific variables
for var in ['ordinal_cols_all', 'bool_cols_all']:
    if var in locals():
        del globals()[var]

# === STEP 1: Boolean Encoding (2015–2019 only) ===
bool_bases = ['has_cooling', 'has_heat', 'protested']
bool_cols_all = [f"{base}_{year}" for base in bool_bases for year in range(2015, 2020)]

for col in bool_cols_all:
    if col in train_merged.columns:
        mode_val = train_merged[col].mode(dropna=True)[0]
        train_merged[col] = train_merged[col].fillna(mode_val).astype(int)
        test_merged[col] = test_merged[col].fillna(mode_val).astype(int)

# === STEP 2: Ordinal Cleaning and Encoding (2015–2019 only) ===
ordinal_bases = [
    'quality', 'quality_description', 'grade',
    'building_condition', 'physical_condition'
]

ordinal_cols_all = [f"{base}_{year}" for base in ordinal_bases for year in range(2015, 2020)]

# Column-specific replacements
replacement_maps = {
    'quality': {'E': 'D', 'F': 'D', 'X': np.nan, 'None': np.nan},
    'quality_description': {'Poor': 'Very Low', 'None': np.nan},
    'grade': {'X': 'F', 'X-': 'F', 'X+': 'F', 'E': 'D', 'E-': 'D-', 'E+': 'D+', 'None': np.nan},
    'building_condition': {'Very Poor': 'Poor', 'Unsound': 'Poor', 'None': np.nan},
    'physical_condition': {'Very Poor': 'Poor', 'Unsound': 'Poor', 'None': np.nan}
}

# Ordinal category order
ord_categories = {
    'quality': ['D', 'C', 'B', 'A'],
    'quality_description': ['Very Low', 'Low', 'Average', 'Good', 'Excellent', 'Superior'],
    'grade': ['F', 'D-', 'D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'],
    'building_condition': ['Poor', 'Fair', 'Average', 'Good', 'Very Good', 'Excellent'],
    'physical_condition': ['Poor', 'Fair', 'Average', 'Good', 'Very Good', 'Excellent']
}

# Clean and encode
for base in ordinal_bases:
    for year in range(2015, 2020):
        col = f"{base}_{year}"
        if col in train_merged.columns:
            replacements = replacement_maps.get(base, {})
            train_merged[col] = train_merged[col].replace(replacements)
            test_merged[col] = test_merged[col].replace(replacements)

            mode_val = train_merged[col].mode(dropna=True)[0]
            train_merged[col] = train_merged[col].fillna(mode_val)
            test_merged[col] = test_merged[col].fillna(mode_val)

            encoder = OrdinalEncoder(categories=[ord_categories[base]], handle_unknown='use_encoded_value', unknown_value=-1)
            train_merged[[col]] = encoder.fit_transform(train_merged[[col]])
            test_merged[[col]] = encoder.transform(test_merged[[col]])



/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2484106990.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_merged[col] = train_merged[col].fillna(mode_val).astype(int)
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2484106990.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_merged[col] = test_merged[col].fillna(mode_val).astype(int)
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/2484106990.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated

In [ ]:
# === STEP 3: Target Encoding (2015–2019 only) ===
def group_and_target_encode_cv(train_df, test_df, target_name, column, rare_threshold=0.001, smoothing=10, n_splits=5):
    freq = train_df[column].value_counts(normalize=True)
    rare_cats = freq[freq < rare_threshold].index
    train_df[column] = train_df[column].replace(rare_cats, 'Other')
    test_df[column] = test_df[column].replace(rare_cats, 'Other')

    global_mean = train_df[target_name].mean()
    oof_encoded = pd.Series(index=train_df.index, dtype='float64')

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    for train_idx, val_idx in kf.split(train_df):
        X_tr, X_val = train_df.iloc[train_idx], train_df.iloc[val_idx]
        stats = X_tr.groupby(column)[target_name].agg(['mean', 'count'])
        smooth = (stats['mean'] * stats['count'] + global_mean * smoothing) / (stats['count'] + smoothing)
        oof_encoded.iloc[val_idx] = X_val[column].map(smooth).fillna(global_mean)

    final_stats = train_df.groupby(column)[target_name].agg(['mean', 'count'])
    final_smooth = (final_stats['mean'] * final_stats['count'] + global_mean * smoothing) / (final_stats['count'] + smoothing)
    test_encoded = test_df[column].map(final_smooth).fillna(global_mean)

    return oof_encoded, test_encoded

# Target-encodable nominal columns
target_encodable_bases = ['foundation_type', 'exterior_walls']
target_encodable_cols_all = [f"{base}_{year}" for base in target_encodable_bases for year in range(2015, 2020)]

# Apply target encoding
for col in target_encodable_cols_all:
    if col in train_merged.columns:
        mode_val = train_merged[col].mode(dropna=True)[0]
        train_merged[col] = train_merged[col].fillna(mode_val)
        test_merged[col] = test_merged[col].fillna(mode_val)

        train_merged[f'{col}_te'], test_merged[f'{col}_te'] = group_and_target_encode_cv(
            train_merged, test_merged, target_name='assessed_2018', column=col,
            rare_threshold=0.001, smoothing=10, n_splits=5
        )

        train_merged.drop(columns=[col], inplace=True)
        test_merged.drop(columns=[col], inplace=True)

print(" Done: Boolean, Ordinal, and Target Encoding for 2015–2019 features only.")

✅ Done: Boolean, Ordinal, and Target Encoding for 2015–2019 features only.


In [ ]:
# Step 1: Get growth columns from training set
growth_cols = [col for col in train_merged.columns if '_growth' in col]

# Step 2: Compute medians from train_merged only
growth_medians = {col: train_merged[col].median() for col in growth_cols}

# Step 3: Apply to both train and test
for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    for col in growth_cols:
        if col in df.columns:
            df[f'{col}_missing'] = df[col].isna().astype(int)
            df[col].fillna(growth_medians[col], inplace=True)
    print(f" Filled and flagged missing values in {df_name} for: {growth_cols}")


✅ Filled and flagged missing values in train_merged for: []
✅ Filled and flagged missing values in test_merged for: []


In [ ]:

# === Step 1: List your growth features ===
growth_features = ['land_value_growth', 'building_value_growth', 'assessed_growth']

# === Step 2: Binning Function (train-based binning) ===
def bin_growth_feature_safe(train_df, test_df, feature, bins=4):
    try:
        # Quantile binning on train only
        train_df[f'{feature}_bin'], bin_edges = pd.qcut(train_df[feature], q=bins, labels=False, retbins=True, duplicates='drop')
        test_df[f'{feature}_bin'] = pd.cut(test_df[feature], bins=bin_edges, labels=False, include_lowest=True)
    except ValueError:
        # Fallback: Equal-width binning
        min_val = train_df[feature].min()
        max_val = train_df[feature].max()
        bin_edges = np.linspace(min_val, max_val, bins + 1)
        train_df[f'{feature}_bin'] = pd.cut(train_df[feature], bins=bin_edges, labels=False, include_lowest=True)
        test_df[f'{feature}_bin'] = pd.cut(test_df[feature], bins=bin_edges, labels=False, include_lowest=True)

    # Convert to category
    train_df[f'{feature}_bin'] = train_df[f'{feature}_bin'].astype('category')
    test_df[f'{feature}_bin'] = test_df[f'{feature}_bin'].astype('category')
    return train_df, test_df

# === Step 3: Apply to train_merged and test_merged ===
for feature in growth_features:
    train_merged, test_merged = bin_growth_feature_safe(train_merged, test_merged, feature)

# === Step 4: Bin year_built_final using train-based quantiles ===
train_merged['year_built_bin'], bin_edges = pd.qcut(
    train_merged['year_built_final'], q=5, retbins=True, labels=False, duplicates='drop'
)
test_merged['year_built_bin'] = pd.cut(
    test_merged['year_built_final'], bins=bin_edges, labels=False, include_lowest=True
)

# Convert to category
train_merged['year_built_bin'] = train_merged['year_built_bin'].astype('category')
test_merged['year_built_bin'] = test_merged['year_built_bin'].astype('category')

# === Step 5: Drop original continuous columns ===
cols_to_drop = growth_features + ['year_built_final']
train_merged.drop(columns=cols_to_drop, inplace=True)
test_merged.drop(columns=cols_to_drop, inplace=True)

print(" Binned growth & year_built features safely with no leakage.")



✅ Binned growth & year_built features safely with no leakage.


In [ ]:
rare_threshold = 0.001  # 0.1%

cat_cols = [
    "quality_description_final",
    "foundation_type_final_te",
    "physical_condition_final",
    "exterior_walls_final_te",
    "region_freq",
    "neighborhood_freq"
]

for col in cat_cols:
    if col in train_merged.columns:
        freq = train_merged[col].value_counts(normalize=True)
        rare = freq[freq < rare_threshold]
        if not rare.empty:
            print(f"\n Rare categories in '{col}' (less than 0.1% of training data):\n{rare}")
    else:
        print(f" Column '{col}' not found in train_merged")




⚠️ Column 'quality_description_final' not found in train_merged
⚠️ Column 'foundation_type_final_te' not found in train_merged
⚠️ Column 'physical_condition_final' not found in train_merged
⚠️ Column 'exterior_walls_final_te' not found in train_merged

⚠️ Rare categories in 'region_freq' (less than 0.1% of training data):
region_freq
0.000963    0.000963
0.000641    0.000641
0.000347    0.000347
0.000159    0.000159
0.000083    0.000083
0.000010    0.000019
0.000002    0.000019
0.000008    0.000016
0.000005    0.000005
Name: proportion, dtype: float64

⚠️ Rare categories in 'neighborhood_freq' (less than 0.1% of training data):
neighborhood_freq
0.000500    0.001000
0.000996    0.000996
0.000988    0.000988
0.000984    0.000984
0.000492    0.000984
              ...   
0.000008    0.000024
0.000021    0.000021
0.000014    0.000014
0.000002    0.000006
0.000003    0.000006
Name: proportion, Length: 321, dtype: float64


In [ ]:
# Define frequency columns and threshold
freq_cols = ['region_freq', 'neighborhood_freq', 'zone_freq', 'subneighborhood_freq']
rare_thresh = 0.001

# Apply rare value replacement for each frequency column
for col in freq_cols:
    if col in train_merged.columns:
        rare_vals = train_merged[col].value_counts(normalize=True)[lambda x: x < rare_thresh].index
        train_merged[col] = train_merged[col].replace(rare_vals, 0)
        test_merged[col] = test_merged[col].replace(rare_vals, 0)
        print(f" Replaced rare values in {col} using train_merged threshold < {rare_thresh}")
    else:
        print(f" Column {col} not found in train_merged — skipping.")



✅ Replaced rare values in region_freq using train_merged threshold < 0.001
✅ Replaced rare values in neighborhood_freq using train_merged threshold < 0.001
✅ Replaced rare values in zone_freq using train_merged threshold < 0.001
✅ Replaced rare values in subneighborhood_freq using train_merged threshold < 0.001


In [ ]:
import numpy as np

# === Step 1: Skew-based Log Transformation (2015–2019 only) ===
log_bases = [
    'floor_area_total', 'porch_area', 'building_area', 'land_area',
    'building_value', 'land_value', 'assessed'
]
neigh_stat_cols = [
    'neigh_assess_mean', 'neigh_assess_std', 'neigh_assess_median',
    'neigh_assess_q1', 'neigh_assess_q3'
]

# Collect log-transformable columns (2015–2019 + neighborhood stats)
log_transform_cols = [f"{base}_{year}" for base in log_bases for year in range(2015, 2020)]
log_transform_cols += neigh_stat_cols

# Compute skewness on train and apply log1p only if skew > 2
for col in log_transform_cols:
    if col in train_merged.columns:
        skew = train_merged[col].skew()
        if skew > 2:
            for df in [train_merged, test_merged]:
                df[f"log_{col}"] = np.log1p(df[col])
            print(f" Log-transformed: {col} (skew={skew:.2f})")
        else:
            print(f"ℹ Skipped: {col} (skew={skew:.2f})")

# === Step 2: Percentile Clipping at 0.1%–99.9% ===
clip_bases = ['floors', 'full_bath', 'total_rooms', 'bedrooms']
clip_cols = [f"{base}_{year}" for base in clip_bases for year in range(2015, 2020)]
clip_cols += ['neigh_count']  # add any other specific columns if needed

# Compute clipping bounds from train
clip_bounds = {
    col: (
        train_merged[col].quantile(0.001),
        train_merged[col].quantile(0.999)
    )
    for col in clip_cols if col in train_merged.columns
}

# Apply clipping to both train and test
for col, (lower, upper) in clip_bounds.items():
    for df in [train_merged, test_merged]:
        if col in df.columns:
            df[col] = np.clip(df[col], lower, upper)
    print(f" Clipped: {col} to [{lower:.2f}, {upper:.2f}]")

print("\n Finished: Skew-aware log transformation + 0.001–0.999 percentile clipping.")


📉 Log-transformed: floor_area_total_2015 (skew=2.22)
📉 Log-transformed: floor_area_total_2016 (skew=2.37)
📉 Log-transformed: floor_area_total_2017 (skew=2.62)
📉 Log-transformed: floor_area_total_2018 (skew=2.83)
📉 Log-transformed: floor_area_total_2019 (skew=3.06)
📉 Log-transformed: porch_area_2015 (skew=4.95)
📉 Log-transformed: porch_area_2016 (skew=4.87)
📉 Log-transformed: porch_area_2017 (skew=4.84)
📉 Log-transformed: porch_area_2018 (skew=4.75)


/usr/local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


📉 Log-transformed: porch_area_2019 (skew=4.67)
📉 Log-transformed: building_area_2015 (skew=2.75)
📉 Log-transformed: building_area_2016 (skew=2.76)
📉 Log-transformed: building_area_2017 (skew=2.82)
📉 Log-transformed: building_area_2018 (skew=2.81)
📉 Log-transformed: building_area_2019 (skew=2.83)
📉 Log-transformed: land_area_2015 (skew=656.66)
📉 Log-transformed: land_area_2016 (skew=560.38)
📉 Log-transformed: land_area_2017 (skew=792.64)
📉 Log-transformed: land_area_2018 (skew=721.01)
📉 Log-transformed: land_area_2019 (skew=501.85)
📉 Log-transformed: building_value_2015 (skew=7.90)
📉 Log-transformed: building_value_2016 (skew=7.79)
📉 Log-transformed: building_value_2017 (skew=7.58)
📉 Log-transformed: building_value_2018 (skew=7.61)
📉 Log-transformed: land_value_2015 (skew=16.10)
📉 Log-transformed: land_value_2016 (skew=15.40)
📉 Log-transformed: land_value_2017 (skew=15.71)
📉 Log-transformed: land_value_2018 (skew=15.18)
📉 Log-transformed: assessed_2015 (skew=10.40)
📉 Log-transformed: as

/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/4250956199.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"log_{col}"] = np.log1p(df[col])
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/4250956199.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"log_{col}"] = np.log1p(df[col])
/var/folders/sl/hc9dmfps60l7lq3bh_fp8ny40000gn/T/ipykernel_32068/4250956199.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

📉 Log-transformed: neigh_assess_median (skew=5.25)
📉 Log-transformed: neigh_assess_q1 (skew=4.89)
📉 Log-transformed: neigh_assess_q3 (skew=5.84)
✂️ Clipped: floors_2015 to [1.00, 4.00]
✂️ Clipped: floors_2016 to [1.00, 4.00]
✂️ Clipped: floors_2017 to [1.00, 4.00]
✂️ Clipped: floors_2018 to [1.00, 4.00]
✂️ Clipped: floors_2019 to [1.00, 4.00]
✂️ Clipped: full_bath_2015 to [1.00, 6.00]
✂️ Clipped: full_bath_2016 to [1.00, 6.00]
✂️ Clipped: full_bath_2017 to [1.00, 6.00]
✂️ Clipped: full_bath_2018 to [1.00, 7.00]
✂️ Clipped: full_bath_2019 to [1.00, 7.00]
✂️ Clipped: total_rooms_2015 to [2.00, 17.00]
✂️ Clipped: total_rooms_2016 to [2.00, 17.00]
✂️ Clipped: total_rooms_2017 to [2.00, 17.00]
✂️ Clipped: total_rooms_2018 to [3.00, 17.00]
✂️ Clipped: total_rooms_2019 to [3.00, 17.00]
✂️ Clipped: bedrooms_2015 to [1.00, 8.00]
✂️ Clipped: bedrooms_2016 to [1.00, 8.00]
✂️ Clipped: bedrooms_2017 to [1.00, 8.00]
✂️ Clipped: bedrooms_2018 to [1.00, 8.00]
✂️ Clipped: bedrooms_2019 to [1.00, 8.00]


In [51]:
def add_features(df):
    df = df.copy()

    # === Ratio features ===
    df['area_ratio'] = df['building_area_2019'] / (df['land_area_2019'] + 1)
    df['porch_ratio'] = df['porch_area_2019'] / (df['building_area_2019'] + 1)
    df['floor_density'] = df['floor_area_total_2019'] / (df['land_area_2019'] + 1)
    df['log_build_density'] = df['log_building_area_2019'] - df['log_land_area_2019']
    df['log_land_to_build_ratio'] = df['log_land_area_2019'] - df['log_building_area_2019']

    df['value_ratio'] = df['building_value_2018'] / (df['land_value_2018'] + 1)
    df['log_value_diff'] = df['log_building_value_2018'] - df['log_land_value_2018']
    df['value_per_sqft'] = df['building_value_2018'] / (df['building_area_2019'] + 1)
    df['price_per_sqft'] = df['assessed_2018'] / (df['building_area_2019'] + 1)

    # === Bathroom & room structure ===
    df['bathroom_score'] = df['full_bath_2019'] + 0.5 * df['half_bath_2019']
    df['bathroom_density'] = df['bathroom_score'] / (df['total_rooms_2019'] + 1)
    df['bedroom_ratio'] = df['bedrooms_2019'] / (df['total_rooms_2019'] + 1)
    df['rooms_per_floor'] = df['total_rooms_2019'] / (df['floors_2019'] + 1)

    # === Core interactions ===
    df['bedrooms_x_floors'] = df['bedrooms_2019'] * df['floors_2019']
    df['rooms_x_quality'] = df['total_rooms_2019'] * df['quality_2019']
    df['log_area_x_grade'] = df['log_building_area_2019'] * df['grade_2019']
    df['log_assess_x_age'] = df['log_assessed_2018'] * df['building_age']
    df['assess_spread_neigh'] = df['log_neigh_assess_q3'] - df['log_neigh_assess_q1']
    df['grade_quality_index'] = df['grade_2019'] * df['quality_2019']

    # === Clean up ===
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(0)
    return df

# === Apply to train and test ===
train_merged = add_features(train_merged)
test_merged = add_features(test_merged)


In [ ]:
# Select numeric columns only
numeric_features = train_merged.select_dtypes(include=['int64', 'float64'])

# Calculate skewness
skewness = numeric_features.skew(numeric_only=True)

# Filter and sort features with skewness > 30
highly_skewed = skewness[skewness > 2].sort_values(ascending=False)

print(" Features with skewness > 2:")
print(highly_skewed)



📊 Features with skewness > 2:
land_area_2017             792.639441
land_area_2018             721.010870
land_area_2015             656.661667
land_area_2016             560.378106
value_ratio                502.848761
                              ...    
floor_area_primary_2018      2.664604
floor_area_total_2017        2.621085
floor_area_primary_2017      2.414200
floor_area_total_2016        2.372924
floor_area_total_2015        2.224773
Length: 76, dtype: float64


In [ ]:
# List of columns to drop due to extreme skewness
cols_to_drop = ['building_value_growth_missing', 'land_value_growth_missing','assessed_growth_missing']

# Drop from both train and test
train_merged.drop(columns=cols_to_drop, inplace=True, errors='ignore')
test_merged.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print("Dropped extremely skewed columns related to value growth missingness.")


✅ Dropped extremely skewed columns related to value growth missingness.


In [ ]:
import numpy as np

# Features to log-transform due to extreme skew
skewed_cols = ['price_per_sqft', 'value_per_sqft', 'porch_ratio', 'value_ratio','land_area_2015','land_area_2016','land_area_2017','land_area_2018','land_area_2019']

for df_name, df in [('train_merged', train_merged), ('test_merged', test_merged)]:
    for col in skewed_cols:
        if col in df.columns:
            df[f'log_{col}'] = np.log1p(df[col])
            df.drop(columns=col, inplace=True)

print("Log-transformed and dropped raw versions of highly skewed ratio features.")


✅ Log-transformed and dropped raw versions of highly skewed ratio features.


In [57]:
if "TARGET" in train_merged.columns:
    train_merged.drop("TARGET", axis=1,inplace=True)
else:
    print("TARGET not found in columns:", train_merged.columns.tolist())

In [ ]:
# Define columns to drop
year_built_cols = [f'year_built_{year}' for year in range(2015, 2020)]

# Drop if columns exist
train_merged.drop(columns=[col for col in year_built_cols if col in train_merged.columns], inplace=True)
test_merged.drop(columns=[col for col in year_built_cols if col in test_merged.columns], inplace=True)

print("Dropped year_built_2015 to year_built_2018 from both train and test sets.")


✅ Dropped year_built_2015 to year_built_2018 from both train and test sets.


In [ ]:
train_cols = set(train_merged.columns)
test_cols = set(test_merged.columns)

missing_in_test = train_cols - test_cols
missing_in_train = test_cols - train_cols

print(" Columns in train but not in test:")
print(sorted(missing_in_test))

print("\n Columns in test but not in train:")
print(sorted(missing_in_train))


✅ Columns in train but not in test:
[]

❌ Columns in test but not in train:
[]


In [60]:
# Check if column sets match
print(set(train_merged.columns) == set(test_merged.columns))  # Should be True

# Check if column order matches
print(list(train_merged.columns) == list(test_merged.columns))  # Must also be True


True
True


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# === STEP 2: Prepare training/test matrices ===
X = train_merged.copy()
X_test = test_merged.copy()
y = pd.Series(y_train).values # use raw target (not log)

# === STEP 3: RidgeCV pipeline ===
kf = KFold(n_splits=3, shuffle=True, random_state=42)
alphas = np.logspace(-3, 2, 10)

ridge_oof = np.zeros(len(X))
ridge_test_preds = np.zeros(len(X_test))
best_alphas = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n Fold {fold+1}/5")

    X_train, y_train_fold = X.iloc[train_idx], y[train_idx]
    X_val, y_val = X.iloc[val_idx], y[val_idx]

    model = make_pipeline(
        StandardScaler(),
        RidgeCV(alphas=alphas, cv=3, scoring='neg_root_mean_squared_error')
    )

    model.fit(X_train, y_train_fold)
    ridge_oof[val_idx] = model.predict(X_val)
    ridge_test_preds += model.predict(X_test) / kf.get_n_splits()

    best_alpha = model.named_steps['ridgecv'].alpha_
    best_alphas.append(best_alpha)
    
    fold_rmse = root_mean_squared_error(y_val, ridge_oof[val_idx])
    print(f"Fold {fold+1} RMSE: {fold_rmse:,.2f} | Best alpha: {best_alpha:.4f}")

# === STEP 4: Final RMSE ===
final_rmse = root_mean_squared_error(y, ridge_oof)
print(f"\n Final OOF RMSE (RidgeCV): {final_rmse:,.2f}")
print(f" Average best alpha across folds: {np.mean(best_alphas):.4f}")

# === STEP 5: Save predictions ===
submission = pd.DataFrame({
    "ACCOUNT": acct_test.values.ravel(),
    "TARGET": ridge_test_preds
})
submission.to_csv("submission_ridgecv_pipeline.csv", index=False)
print("\n Saved: submission_ridgecv_pipeline.csv")

# === Optional: Save OOF & test preds for stacking or analysis ===
np.save("ridgecv_oof_preds.npy", ridge_oof)
np.save("ridgecv_test_preds.npy", ridge_test_preds)
print(" Saved: ridgecv_oof_preds.npy and ridgecv_test_preds.npy")



📂 Fold 1/5
📉 Fold 1 RMSE: 42,050.33 | Best alpha: 2.1544

📂 Fold 2/5
📉 Fold 2 RMSE: 41,036.52 | Best alpha: 27.8256

📂 Fold 3/5
📉 Fold 3 RMSE: 40,619.40 | Best alpha: 0.5995

🏁 Final OOF RMSE (RidgeCV): 41,239.79
🔍 Average best alpha across folds: 10.1932

📤 Saved: submission_ridgecv_pipeline.csv
📦 Saved: ridgecv_oof_preds.npy and ridgecv_test_preds.npy
